In [1]:
import os
os.chdir("/home/anderson/Documents/word2vec/")
os.getcwd()

'/home/anderson/Documents/word2vec'

In [2]:
import pandas as pd

In [3]:
data_keys= pd.read_csv("data_qry.csv", header=5, delimiter=",")

In [4]:
data_sub= pd.DataFrame(data_keys, columns=["Search term","Match type"])

In [6]:
data_sub.columns=["search_term", "match_type"]

In [87]:
data_extract= data_sub[(data_sub['match_type']=='broad')]

In [101]:
data_extract=data_extract.reset_index(drop=True)
data_extract.head()

,search_term,match_type
0,victoria falls ausflugsmöglichkeiten,broad
1,malaga concerts excursions etc en novembre 2014,broad
2,welcome cardkeln,broad
3,new york travel guide foot,broad
4,museum tour wien,broad


In [45]:
from bs4 import BeautifulSoup
import re

In [124]:
def clean_words(dirty_words):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove square brackets
    square_removal = BeautifulSoup(dirty_words).get_text()
    #  
    # 2. Remove non-letters
    text_only = re.sub("[^a-zA-Z]"," ", square_removal)
    #
    # 3. Convert words to lower case and split them
    words = text_only.lower().split()
    #
    # 4. Create list of words
    word_list = [w for w in words]
    #
    # 5. Return a list of lists
    return(word_list)

def clean_phrases(dirty_words):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove square brackets
    square_removal = BeautifulSoup(dirty_words).get_text()
    #  
    # 2. Remove non-letters
    text_only = re.sub("[^a-zA-Z]","_", square_removal)
    #
    # 3. Convert words to lower case and split them
    words = text_only.lower().split()
    #
    # 4. Create list of words
    word_list = [w for w in words]
    #
    # 5. Return a list of lists
    return(word_list)

In [104]:
sub= data_extract.search_term[0]
print sub

victoria falls ausflugsmöglichkeiten


In [109]:
# Get the number of keyword entries based on the dataframe column size
num_keywords = len(data_extract)
# Initialize an empty list to hold the tokenized keywords
tokenized_keys = []

# Loop over each review; create an index i that goes from 0 to the length
# of the keyword list 
for i in xrange(0, num_keywords):
    # Call our function for each one, and add the result to the list of
    # tokenized keywords
    tokenized_keys.append( clean_words(data_extract.search_term[i]))

In [111]:
tokenized_keys[0]

[u'victoria', u'falls', u'ausflugsm', u'glichkeiten']

In [112]:
import logging
logging.basicConfig(format='%(asctime)s:%(levelname)s:%(messages)s', level= logging.INFO)

In [113]:
num_features=300
min_word_count=40
num_workers=4
context=10
downsampling= 1e-3

In [114]:
from gensim.models import word2vec

In [115]:
print "Training model"
model= word2vec.Word2Vec(tokenized_keys, workers=num_workers, size=num_features, min_count= min_word_count,\
window= context, sample= downsampling)
model.init_sims(replace=True)
model_name= "300feature_search_terms_broad"
model.save(model_name)

Training model


In [116]:
model.most_similar("")

[(u'gide', 0.6543757915496826),
 (u'guides', 0.6310587525367737),
 (u'find', 0.6148226261138916),
 (u'our', 0.5780949592590332),
 (u'guid', 0.5697128772735596),
 (u'your', 0.5556690096855164),
 (u'tourguide', 0.5499275326728821),
 (u'way', 0.54803466796875),
 (u'get', 0.5471200942993164),
 (u'russian', 0.5435190796852112)]

In [127]:
# Get the number of keyword entries based on the dataframe column size
num_keywords = len(data_extract)
# Initialize an empty list to hold the tokenized keywords
key_phrases= []

# Loop over each review; create an index i that goes from 0 to the length
# of the keyword list 
for i in xrange(0, num_keywords):
    # Call our function for each one, and add the result to the list of
    # tokenized keywords
    key_phrases.append(clean_phrases(data_extract.search_term[i]))

In [136]:
tokenized_keys[0]

[u'victoria', u'falls', u'ausflugsm', u'glichkeiten']

In [137]:
import gensim as gs

In [138]:
ngrams= gs.models.phrases.Phrases(tokenized_keys,min_count=5,\
        threshold=10.0, delimiter='_')

In [141]:
from gensim.models.phrases import Phrases

In [152]:
bigram_transformer = Phrases(key_phrases)

In [155]:
model= word2vec.Word2Vec.load("300feature_search_terms_broad")

In [160]:
model.most_similar(positive=['germany', 'berlin'], negative=['france'])

[(u'welcome', 0.5177892446517944),
 (u'abc', 0.4719970226287842),
 (u'berliner', 0.47185343503952026),
 (u'alexanderplatz', 0.46284401416778564),
 (u'bvg', 0.4620282053947449),
 (u'discounts', 0.4544591009616852),
 (u'rabatte', 0.44964665174484253),
 (u'main', 0.4277723729610443),
 (u'lisbon', 0.42146673798561096),
 (u'hamburg', 0.4120604395866394)]

In [168]:
model.similarity('paris','france')

0.70569431527179838

In [179]:
bigram= Phrases(tokenized_keys)

In [191]:
sent = [u'hotel',u'london', u'tower', u'bridge', u'new', u'york', u'germany', u'berlin']

In [192]:
print(bigram[sent])

[u'hotel', u'london', u'tower_bridge', u'new_york', u'germany', u'berlin']


In [193]:
trigram= Phrases(bigram[sent])

In [194]:
print(trigram[bigram[sent]])

[u'hotel', u'london', u'tower_bridge', u'new_york', u'germany', u'berlin']
